In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
from torch.utils.data import DataLoader

import json

from tqdm import tqdm_notebook as tqdm

from my_imp.models.nscl import NSCLModel
from my_imp.copied.scene_annotation import annotate_objects
from my_imp.data import MyDataset, gen_image_transform, gen_bbox_transform

12 13:45:29 Loading c extension from: "/Users/sebamenabar/Documents/TAIA/Jacinle/jaclearn/vision/coco".


In [63]:
nscl

NSCLModel(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stat

In [8]:
img_size = 256

ans_dict_json = 'data/clevr/CLEVR_built_dictionaries.json'

image_root = 'data/clevr/train/images/'
vocab_json = 'data/clevr/train/vocab.json'
train_scenes_json = 'data/clevr/train/scenes.json'
train_questions_json = 'data/clevr/train/questions.json'

val_image_root = 'data/clevr/val/images/'
val_vocab_json = 'data/clevr/val/vocab.json'
val_scenes_json = 'data/clevr/val/scenes.json'
val_questions_json = 'data/clevr/val/questions.json'

dataset_args = {
    'train': {
        'scenes_json': train_scenes_json,
        'questions_json': train_questions_json,
        'image_root': image_root,
        'vocab_json': vocab_json,
    },
    'val': {
        'scenes_json': val_scenes_json,
        'questions_json': val_questions_json,
        'image_root': val_image_root,
        'vocab_json': val_vocab_json,
    },
}

image_transform = gen_image_transform(img_size)
bbox_transform = gen_bbox_transform(img_size)

In [18]:
split = 'val'

my_dataset = MyDataset(
    image_transform=image_transform,
    bbox_transform=bbox_transform,
    ans_dict_json=ans_dict_json,
    **dataset_args[split],
)

Loading answers from: "data/clevr/CLEVR_built_dictionaries.json"
Loading scenes from: "data/clevr/val/scenes.json".
Loading questions from: "data/clevr/val/questions.json".
Loading vocab from: "data/clevr/val/vocab.json".
Preparing scenes
15000/15000
Preparing questions
149991/149991


In [11]:
dato = my_dataset[0]

In [12]:
dato['program_raw'] = dato['program']

In [65]:
from my_imp.data import collate_fn

In [66]:
loader = DataLoader(my_dataset, batch_size=2, num_workers=2, shuffle=True, collate_fn=collate_fn)

In [67]:
batch = next(iter(loader))

In [72]:
nscl.resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [74]:
nscl = NSCLModel()
nscl(batch)

torch.Size([2, 98304])
Using /private/var/folders/3l/bh48rdbn141c_fvqn5vf6whh0000gn/T/torch_extensions as PyTorch extensions root...
Creating extension directory /private/var/folders/3l/bh48rdbn141c_fvqn5vf6whh0000gn/T/torch_extensions/_prroi_pooling...
Detected CUDA files, patching ldflags


OSError: CUDA_HOME environment variable is not set. Please set it to your CUDA install root.

In [39]:
from tqdm import tqdm_notebook as tqdm

In [ ]:
for batch in tqdm(loader):
    pass

In [32]:
print(batch[0][0].size())
print(batch[0][1].size())
print(batch[0][2].size())
print(batch[0][3].size())
print(batch[1].size())


torch.Size([2, 3, 256, 384])
torch.Size([2, 7, 4])
torch.Size([2])
torch.Size([2, 32])
torch.Size([2])


In [29]:
import sys

sys.path.insert(0, '../NSCL-PyTorch-Release/')

In [60]:
from nscl.datasets.datasets import NSCLDataset

In [78]:
NSCLDataset(val_scenes_json, val_questions_json, image_root=val_image_root, vocab_json=val_vocab_json, image_transform=img_transform)

10 23:18:09 Loading scenes from: "data/clevr/val/scenes.json".
10 23:18:11 Loading questions from: "data/clevr/val/questions.json".
Exception ignored in: <function _DataLoaderIter.__del__ at 0x126e03378>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 677, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 659, in _shutdown_workers
    w.join()
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 140, in join
    res = self._popen.wait(timeout)
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/popen_fork.py", line 48, in wait
    return self.poll(os.WNOHANG if timeout == 0.0 else 0)
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/popen_fork.py", lin

In [41]:
torch.cuda.is_available()

False